In [13]:
import cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import rospy

import pyrealsense2
from realsense_depth import *
import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
from open3d_ros_helper import open3d_ros_helper as orh
from scipy import spatial
from scipy.spatial.transform import Rotation as R
import math
import tf2_ros, tf
import geometry_msgs.msg
import PyKDL
import time

image = None

rospy.init_node("my_pic", anonymous=True)

tfbuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfbuffer)
br = tf2_ros.TransformBroadcaster()
t = geometry_msgs.msg.TransformStamped()

bridge = CvBridge()
loop_rate = rospy.Rate(0.5) # Node cycle rate (in Hz).
np.set_printoptions(suppress=True)

In [14]:
def grab_frame():
    
    frame_color=rospy.wait_for_message('/camera/color/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_color = bridge.imgmsg_to_cv2(frame_color, desired_encoding='rgb8')
    
    frame_depth = rospy.wait_for_message('/camera/depth/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_depth = bridge.imgmsg_to_cv2(frame_depth)
    
    return cv_image_color, cv_image_depth

def visualize_rgbd(rgbd_image):
    print(rgbd_image)
    #o3d.visualization.draw_geometries([rgbd_image])
    
    #intrinsic = o3d.camera.PinholeCameraIntrinsic()
    #intrinsic.intrinsic_matrix =  [[462.1379699707031, 0.0, 320.0], [0.0, 462.1379699707031, 240.0], [0.0, 0.0, 1.0]]
    #intrinsic.intrinsic_matrix =  [[347.99755859375, 0.0, 320.0], [0.0, 347.99755859375, 240.0], [0.0, 0.0, 1.0]]
    #intrinsic.intrinsic_matrix =  [[602.71783447, 0.0, 313.06835938], [0.0, 601.61364746, 230.37461853], [0.0, 0.0, 1.0]]
    
    #w = 640
    #h = 480
    #fx = 602.71783447
    #fy = 601.61364746
    #cx = 313.06835938
    #cy = 230.37461853
    
    #Color frame ROS camera
    #w = 640
    #h = 480
    #fx = 462.1379699707031
    #fy = 462.1379699707031
    #cx = 320.0
    #cy = 240.0
    
    #Depth frame ROS camera
    w = 640
    h = 480
    fx = 347.99755859375
    fy = 347.99755859375
    cx = 320.0
    cy = 240.0    
    
    
    intrinsic = o3d.camera.PinholeCameraIntrinsic(w, h, fx,fy, cx, cy)
    intrinsic.intrinsic_matrix = [[fx, 0, cx], [0, fy, cy], [0, 0, 1]]
    
    cam = o3d.camera.PinholeCameraParameters()
    cam.intrinsic = intrinsic
    
    #cam.extrinsic = np.array([[0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 1.]])
    #pcd = o3d.geometry.create_point_cloud_from_rgbd_image(rgbd_image, cam.intrinsic, cam.extrinsic)
    
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic)
    #pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,intrinsic)
    
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down.
    
    #o3d.visualization.draw_geometries([pcd])
    
    return pcd
    
    
def tst_dataset(color_frame,depth_frame):
    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)) )
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
    pcd = visualize_rgbd(rgbd_image)
    return pcd


def box_pos(x_coord, y_coord, width, height, centered=0):
    if centered == 0:
        start_point = (x_coord, y_coord) # represents the top left corner of rectangle
        end_point = (x_coord+width-1, y_coord+height-1)  # represents the bottom right corner of rectangle
    elif centered == 1:
        new_x = x_coord - (np.floor(width/2)-1).astype(int)
        new_y = y_coord - (np.floor(height/2)-1).astype(int)
        start_point = (new_x, new_y) 
        end_point = (new_x+width, new_y+height)
    return start_point, end_point


def fetch_transform(tfbuffer,frame1,frame2,quat=0):
    flag = 0
    while flag==0:
        try:
            trans = tfbuffer.lookup_transform(frame1, frame2, rospy.Time(),rospy.Duration(8.0))
            #print (trans)
            trans = trans.transform  #save translation and rotation
            #rot = PyKDL.Rotation.Quaternion(* [ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w] )
            #ypr = [ i  / np.pi * 180 for i in rot.GetEulerZYX() ]
            #print(ypr[2],ypr[1],ypr[0])
            
            rot = R.from_quat([ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w]) #creates rotation matrix
            rpy = rot.as_euler('XYZ',degrees=True) #extrinsic
            break
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException) as e:
            #print ("Fail", e)
            continue
    if quat==0:
        return trans.translation.x, trans.translation.y, trans.translation.z, rpy[0],rpy[1],rpy[2] #ypr[2], ypr[1], ypr[0]
    elif quat==1:
        return trans.translation.x, trans.translation.y, trans.translation.z, trans.rotation.x, trans.rotation.y, trans.rotation.z, trans.rotation.w
    


def Axis_angle_to_Quat(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    qx = x * np.sin(angle/2)
    qy = y * np.sin(angle/2)
    qz = z * np.sin(angle/2)
    qw = np.cos(angle/2)
    
    return qx,qy,qz,qw

def Axis_angle_to_Euler(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    s=np.sin(angle)
    c=np.cos(angle)
    t=1-c
    
    if ((x*y*t + z*s) > 0.998):  #north pole singularity detected
        
        heading = 2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = np.pi/2
        bank = 0
        return heading, attitude, bank
    
    elif ((x*y*t + z*s) < -0.998):
        
        heading = -2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = -np.pi/2
        bank = 0
        return heading, attitude, bank
    
    heading = np.arctan2(y * s- x * z * t , 1 - (y*y+ z*z ) * t)
    attitude = np.arcsin(x * y * t + z * s)
    bank = np.arctan2(x * s - y * z * t , 1 - (x*x + z*z) * t)
    
    return bank, heading, attitude


def getRotation2(v1):
    if np.all(v1==[0., 0., 1.]): v1 = [0, 0.000001, 0.999999]
    vec_x = [1.,0.,0.] 
    vec_y = [0.,1.,0.]
    vec_z = [0.,0.,1.] #>>>>>>>>>>>>>>>>>>>>> change these to -1 to flip again
    
    vec1 = v1 / np.linalg.norm(v1)

    #vector_x = np.cross(vec1, vec_x)/np.linalg.norm(np.cross(vec1, vec_x))
    angle_x = (math.acos(np.dot(vec1, vec_x)))

    #vector_y = np.cross(vec1, vec_y)/np.linalg.norm(np.cross(vec1, vec_y))
    angle_y = (math.acos(np.dot(vec1, vec_y)))

    vector_z = np.cross(vec1, vec_z)/np.linalg.norm(np.cross(vec1, vec_z))
    angle_z = (math.acos(np.dot(vec1, vec_z)))

    #Rotation = filtered_Pt_Cloud.get_rotation_matrix_from_axis_angle(angle*vector) #alternative Open3D lib.
    Rotation = R.from_rotvec(angle_z*vector_z)
    
    return Rotation, angle_x, angle_y, angle_z


def Bounds_gen(minB, maxB, spacing):
    UpperB = 0
    ctr = 0
    bounds = np.array([[0.,0.]])
    while UpperB < maxB: #from left most to right most

        LowerB = minB + ctr * spacing #if we shift the X or Y coordinates in multiples of spacing, we should get different lines.
        if ctr == 0:
            UpperB = LowerB + spacing/2 #lower + spacing/2 only for first condition
        else:
            UpperB = LowerB + spacing #lower + spacing
        bounds = np.append(bounds,[[LowerB,UpperB]], axis=0)
        ctr+= 1
    bounds = np.delete(bounds, 0, axis=0)
    return bounds

def cropped_PC(original_PC, spacing, X = 1, idx = 0, centered = True, Sorted = True):

    PC_BBOX = original_PC.get_axis_aligned_bounding_box()
    minB_X = PC_BBOX.min_bound[0]
    maxB_X = PC_BBOX.max_bound[0]
    minB_Y = PC_BBOX.min_bound[1]
    maxB_Y = PC_BBOX.max_bound[1]
    minB_Z = PC_BBOX.min_bound[2]
    maxB_Z = PC_BBOX.max_bound[2]
    
    if X == 1: #X sided sweep crop
        bounds = Bounds_gen(minB_X, maxB_X, spacing)
        print("Total bounds:",len(bounds))
        if centered == True:
            idx = int(np.floor(len(bounds)/2))
        else:
            if idx > len(bounds)-1:
                print("Warning! Index value out of bounds, setting to max value")
                idx = -1
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(bounds[idx][0], minB_Y, minB_Z), max_bound=(bounds[idx][1], maxB_Y, maxB_Z))

        if Sorted:
            result = original_PC.crop(bbox)
            res_pts = np.array(result.points)
            res_nor = np.array(result.normals)

            ind = np.argsort(res_pts[:, 1]) #Sort Y coordinates from lowest to highest, X values are almost constant
            res_pts = res_pts[ind]
            res_nor = res_nor[ind]
            sorted_pointcloud = o3d.geometry.PointCloud()
            sorted_pointcloud.points = o3d.utility.Vector3dVector(res_pts)
            sorted_pointcloud.normals = o3d.utility.Vector3dVector(res_nor)
            return sorted_pointcloud

    elif X == 0: #Y sided sweep crop
        bounds = Bounds_gen(minB_Y, maxB_Y, spacing)
        print("Total bounds:",len(bounds))
        if centered == True:
            idx = int(np.floor(len(bounds)/2))
        else:
            if idx > len(bounds)-1:
                print("Warning! Index value out of bounds, setting to max value")
                idx = -1
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X, bounds[idx][0], minB_Z), max_bound=(maxB_X, bounds[idx][1], maxB_Z)) 
    
        if Sorted:
            result = original_PC.crop(bbox)
            res_pts = np.array(result.points)
            res_nor = np.array(result.normals)

            ind = np.argsort(res_pts[:, 0]) #Sort X coordinates from lowest to highest, Y values are almost constant
            res_pts = res_pts[ind]
            res_nor = res_nor[ind]
            sorted_pointcloud = o3d.geometry.PointCloud()
            sorted_pointcloud.points = o3d.utility.Vector3dVector(res_pts)
            sorted_pointcloud.normals = o3d.utility.Vector3dVector(res_nor)
            return sorted_pointcloud
    
    return original_PC.crop(bbox)

def Publish_coordinates(point_cloud):
    point_cloud_pts = np.array(point_cloud.points)
    point_cloud_nor = np.array(point_cloud.normals)
    
    for index in range (len(res_pts)):

        rotat, angle_x, angle_y, angle_z = getRotation2(point_cloud_nor[index])
        flip_frame = R.from_euler('xyz', [0., 180, 0.], degrees=True) 
        #flip_frame2 = R.from_euler('xyz', [0., 0., 180.], degrees=True) #Use if-else to switch between cases
        #rotat = flip_frame*rotat*flip_frame2  #Use if-else to switch between cases

        rotat = flip_frame*rotat  #to match correct normal/plane pose (flip later to send to robot)
        r_quat = rotat.as_quat()            #x,y,z,w format

        t.header.frame_id = "camera_depth_optical_frame"
        t.child_frame_id = "plane_"+str(index)

        t.header.stamp = rospy.Time.now()
        t.transform.translation.x = point_cloud_pts[index][0]
        t.transform.translation.y = point_cloud_pts[index][1]
        t.transform.translation.z = -point_cloud_pts[index][2] #becaues z in opposite dirn

        #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)

        t.transform.rotation.x = r_quat[0]
        t.transform.rotation.y = r_quat[1]
        t.transform.rotation.z = r_quat[2]
        t.transform.rotation.w = r_quat[3]

        br.sendTransform(t)

In [55]:
color_frame, depth_frame = grab_frame()
print("color frame:",color_frame.shape, " Depth frame:",depth_frame.shape)

#Check center pixel distance

point = (320, 240)

spacing = 0.01 #spacing between each point in point cloud in meters (use this to approximate the shape of surface)

#color_frame, depth_frame = grab_frame()

distance = depth_frame[point[1], point[0]]
print("Center of image:", distance, "mm")

pt_cloud = tst_dataset(color_frame, depth_frame)

downpcd = pt_cloud.voxel_down_sample(voxel_size=spacing)
#o3d.visualization.draw_geometries([downpcd])

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30)) #radius in meters
print("Align normals towards camera")
downpcd.orient_normals_towards_camera_location(camera_location=[0, 0, 0])

o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

#v2_camera = np.array(downpcd.normals)
v2_camera_pts = np.array(downpcd.points)


#idx = np.where(abs(v2_camera_pts[:,2]) < (distance/1000)+0.01)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec
idx = np.where(abs(v2_camera_pts[:,2]) < abs(np.min(v2_camera_pts[:,2]))-0.06)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec

#print(idx)
filtered_Pt_Cloud = downpcd.select_by_index(idx, invert=False)
print(filtered_Pt_Cloud)
o3d.visualization.draw_geometries([filtered_Pt_Cloud], point_show_normal=True)
nor = np.array(filtered_Pt_Cloud.normals)
pts = np.array(filtered_Pt_Cloud.points)

print()
distance,index = spatial.KDTree(pts).query( filtered_Pt_Cloud.get_center() ) #find coordinates 
                                                                             #that are closest to the center 

print(distance)
print(index)
print()
print("Center Coordinates(ground truth):",filtered_Pt_Cloud.get_center())
print("Points Coordinates(estimated center point):",pts[index])
print("Normal Coordinates(Normal of estimated center point):",nor[index])

color frame: (480, 640, 3)  Depth frame: (480, 640)
Center of image: 257 mm
RGBDImage of size 
Color image : 640x480, with 3 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.
Recompute the normal of the downsampled point cloud
Align normals towards camera
PointCloud with 755 points.

0.034497148386625445
433

Center Coordinates(ground truth): [ 0.0132688  -0.02532002 -0.29080169]
Points Coordinates(estimated center point): [ 0.00662912 -0.02716759 -0.257     ]
Normal Coordinates(Normal of estimated center point): [-0.08027963 -0.02130752  0.99654462]


### Create Coordinate pointer mesh

In [56]:
#mesh with coordinate
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
#o3d.visualization.draw_geometries([filtered_Pt_Cloud,mesh], point_show_normal=True)

### Crop point cloud to get a points in a single line in X or Y axis, 
*Use idx to give offset else use centered to get the points from center of the object*

In [58]:
result = cropped_PC(filtered_Pt_Cloud, spacing, X = 0, idx = 0, centered = True, Sorted = True)
o3d.visualization.draw_geometries([result, mesh], point_show_normal=True)

res_pts = np.array(result.points)
res_nor = np.array(result.normals)

Total bounds: 38


### This will boradcast all points in pointcloud to tf2 topic, visualize using Rviz

In [59]:
Publish_coordinates(result)

### Snip to test and visualize the tf2 frame order

In [60]:
for index in range (len(res_pts)):

    rotat, angle_x, angle_y, angle_z = getRotation2(res_nor[index])

    flip_frame = R.from_euler('xyz', [0., 180, 0.], degrees=True) 
    #flip_frame2 = R.from_euler('xyz', [0., 0., 180.], degrees=True) #Use if-else to switch between cases
    #rotat = flip_frame*rotat*flip_frame2  #Use if-else to switch between cases

    rotat = flip_frame*rotat  #to match correct normal/plane pose (flip later to send to robot)

    r = rotat.as_euler("xyz", degrees=True)

    r_quat = rotat.as_quat()            #x,y,z,w format

    print("X axis and normal angle in degrees",np.degrees(angle_x))
    print()
    print("Y axis and normal angle in degrees",np.degrees(angle_y))
    print()
    print("Z axis and normal angle in degrees",np.degrees(angle_z))
    print()

    print("Rotation wrt camera:","Roll:", np.round(r[0],3), "Pitch:", np.round(r[1],3), "Yaw:", np.round(r[2],3))
    #print("In Quaternion form xyzw:",r_quat)
    print()
    print("Coordinates wrt camera:",res_pts[index])

    t.header.frame_id = "camera_depth_optical_frame"
    t.child_frame_id = "plane"

    t.header.stamp = rospy.Time.now()
    t.transform.translation.x = res_pts[index][0]
    t.transform.translation.y = res_pts[index][1]
    t.transform.translation.z = -res_pts[index][2] #becaues z in opposite dirn

    #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)

    t.transform.rotation.x = r_quat[0]
    t.transform.rotation.y = r_quat[1]
    t.transform.rotation.z = r_quat[2]
    t.transform.rotation.w = r_quat[3]

    br.sendTransform(t)
    time.sleep(0.5)

X axis and normal angle in degrees 151.24740918924047

Y axis and normal angle in degrees 90.79685209841035

Z axis and normal angle in degrees 61.26054975483868

Rotation wrt camera: Roll: 178.343 Pitch: -61.247 Yaw: -179.019

Coordinates wrt camera: [-0.0774065  -0.07741042 -0.32454545]
X axis and normal angle in degrees 149.5687497986253

Y axis and normal angle in degrees 91.31105652355997

Z axis and normal angle in degrees 59.60310162028596

Rotation wrt camera: Roll: 177.411 Pitch: -59.569 Yaw: -178.518

Coordinates wrt camera: [-0.07228148 -0.07733215 -0.31246667]
X axis and normal angle in degrees 145.93347024370703

Y axis and normal angle in degrees 90.97905778736634

Z axis and normal angle in degrees 55.95149770749231

Rotation wrt camera: Roll: 178.252 Pitch: -55.933 Yaw: -179.072

Coordinates wrt camera: [-0.06569269 -0.07731306 -0.30089474]
X axis and normal angle in degrees 140.64818502239808

Y axis and normal angle in degrees 91.18782713828402

Z axis and normal angl

X axis and normal angle in degrees 37.15280019875707

Y axis and normal angle in degrees 90.97328102324767

Z axis and normal angle in degrees 52.86437293412021

Rotation wrt camera: Roll: 178.388 Pitch: 52.847 Yaw: 179.199

Coordinates wrt camera: [ 0.09812899 -0.07735991 -0.29961765]
X axis and normal angle in degrees 35.606139180239694

Y axis and normal angle in degrees 90.61712000118601

Z axis and normal angle in degrees 54.40088179037411

Rotation wrt camera: Roll: 178.94 Pitch: 54.394 Yaw: 179.455

Coordinates wrt camera: [ 0.1018795  -0.07729156 -0.30563636]
X axis and normal angle in degrees 33.7094796357832

Y axis and normal angle in degrees 90.5618614371706

Z axis and normal angle in degrees 56.2964876978164

Rotation wrt camera: Roll: 178.988 Pitch: 56.291 Yaw: 179.458

Coordinates wrt camera: [ 0.10544026 -0.07743815 -0.31233333]


### Use this to fetch transform between two frames that are published in tf2

In [62]:
transform_plane = fetch_transform(tfbuffer,'world', 'camera_depth_optical_frame',quat=0)

transform_plane = np.round(transform_plane,3)
print("Trans XYZ:                            ",transform_plane[0], transform_plane[1], transform_plane[2])
print("Rot in world coordinate system (deg):",transform_plane[3],transform_plane[4],transform_plane[5])

Trans XYZ:                             0.303 0.017 0.456
Rot in world coordinate system (deg): 179.947 -1.065 90.001



start_point,end_point = box_pos(320, 240, 10, 10, centered=1)  #x,y,width,height, 0-- top left coord, 1--- center coord

cf = color_frame
for i in range (color_frame.shape[0]):
    for j in range (color_frame.shape[1]):
        if depth_frame[i][j] == 0:
            cf[i][j] = [0 , 0, 0] 

plt.imshow(cf)
plt.show()
window_name = 'Filtered_image'  # Window name in which image is displayed

cv2.rectangle(cf, start_point, end_point, (0, 0, 255), 1)
cv2.imshow(window_name, cf)
cv2.waitKey(0)
cv2.destroyAllWindows()


#color_frame_crop = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]

color_frame_crop = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]
depth_frame_crop = depth_frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

print(depth_frame_crop.shape)
zz=depth_frame_crop.T
print(zz.shape)

#cv2.imshow("cropped", depth_frame_crop)
#cv2.waitKey(0)
#cv2.destroyAllWindows()